In [1]:
#분석용 서버구축을 위한 flask 설치
!pip install flask

In [2]:
!pip install matplotlib

In [3]:
!pip install opencv-python

In [4]:
### 불러오기
from flask import Flask, request,Response,redirect
import io # 파이썬의 입출력을 도와주는 라이브러리 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model # 딥러닝 모델 로딩 함수
import cv2
#from PIL import image , imageOps

In [ ]:
# 플라스크 객체 생성
app = Flask(__name__)
model=load_model('./keras_model.h5')

#사용자 요청을 처리하기 위한 라우터 설정      ##### 라우터 :데이터를 전송할 수 있게하는 장치
@app.route('/predict',methods=['GET','POST'])
def predict():
    if request.method == 'POST':
        print(1)
        img = request.files['file']
        print(request.files)
        input_stream = io.BytesIO() # byte단위로 읽어들이는 통로
        img.save(input_stream) # file로 부터 데이터를 읽기
        data = np.fromstring(input_stream.getvalue(), dtype = np.uint8)
        
        real_img = cv2.imdecode(data, 1) # 1 => 컬러정보
        real_img = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB) # BGR에서 RGB
        resize_img = cv2.resize(real_img, dsize = (224, 224), interpolation=cv2.INTER_AREA)
        
        #display(plt.imshow(resize_img))    ##### 이미지를 파이썬에서 확인하는부분
        #plt.show()
        
        normalized_img=(np.array(resize_img,dtype=np.float32)/127.0  )-1
        
        # 예측시킬 이미지 데이터를 넣을 변수
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        # 불러온 numpy 타입의 이미지를 변수에 대입
        data[0] = normalized_img
        # 예측
        prediction = model.predict(data)
        index = np.argmax(prediction[0]) # 최대값의 인덱스
        
        print(index,'정답')

    return redirect('http://localhost:8090/machine/predict.jsp?pre={}'.format(index) )   ##### 이클립스 실행시켜서 나오는 jsp페이지
        
        
    
app.run()  # 서버구동



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
